In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Case Study: Clinical NLP for Automated Medical Coding
## Implementation Notebook

---

**Scenario:** You are an ML engineer at MedScribe AI, a healthtech startup building clinical documentation intelligence for hospitals. Your task is to build a BERT-based Named Entity Recognition (NER) system that extracts clinical entities (diagnoses, medications, procedures) from unstructured physician notes and maps them to ICD billing codes.

**Current system:** A rule-based NLP pipeline with 68% entity extraction accuracy and 74% negation detection accuracy. Your target: 88%+ entity-level F1.

**Why BERT:** The rule-based system fails on negation ("patient denies chest pain" vs. "patient reports chest pain"), context-dependent terminology, and implicit references. BERT's deep bidirectional self-attention can capture the full context around each word, making it ideal for clinical NER.

---

## 3.1 Data Acquisition and Preprocessing

We use the **BC5CDR dataset** (BioCreative V Chemical Disease Relation) -- a publicly available biomedical NER benchmark with 1,500 annotated PubMed abstracts containing chemical and disease entity annotations. This closely mirrors the structure of clinical NER while being accessible without credentialed access.

The BC5CDR dataset has the same core challenge as MedScribe's clinical notes: extracting biomedical entities from unstructured text where context determines meaning.

In [ ]:
# Install dependencies
!pip install transformers datasets seqeval matplotlib scikit-learn -q

import torch
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from datasets import load_dataset
from transformers import BertTokenizerFast

# Load the BC5CDR dataset (biomedical NER)
dataset = load_dataset("tner/bc5cdr")

# Examine the dataset structure
print("Dataset splits:", dataset)
print("\nSample entry:")
sample = dataset['train'][0]
print("Tokens:", sample['tokens'])
print("Tags:", sample['tags'])

print("\nLabel mapping:")
label_names = dataset['train'].features['tags'].feature.names
for i, name in enumerate(label_names):
    print(f"  {i}: {name}")

# Initialize BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
print(f"\nTokenizer vocabulary size: {tokenizer.vocab_size}")

### TODO 1: Data Exploration

Analyze the BC5CDR dataset to understand its characteristics before building any models.

In [ ]:
def explore_dataset(dataset):
    """
    Analyze the BC5CDR dataset to understand its characteristics.

    Compute and print the following statistics:
    1. Number of examples in train/validation/test splits
    2. Average number of tokens per example
    3. Distribution of entity types (count of B-Chemical, I-Chemical,
       B-Disease, I-Disease, O labels across the training set)
    4. Percentage of tokens that are entity tokens vs. O tokens
    5. Average number of entities per example
    6. Distribution of entity lengths (in tokens)

    Args:
        dataset: HuggingFace dataset with 'tokens' and 'tags' fields

    Returns:
        dict with keys: 'split_sizes', 'avg_tokens', 'label_distribution',
              'entity_percentage', 'avg_entities', 'entity_lengths'

    Hints:
    - Use dataset['train'].features['tags'].feature.names to get label names
    - An entity starts at any B- tag and continues through consecutive I- tags
    - To count entities, iterate through tags and count B- tag occurrences
    """
    # YOUR CODE HERE
    pass

stats = explore_dataset(dataset)

In [ ]:
# Verification: Check your implementation
assert 'split_sizes' in stats, "Missing 'split_sizes' key"
assert 'label_distribution' in stats, "Missing 'label_distribution' key"
assert stats['split_sizes']['train'] > 0, "Training set should not be empty"
assert stats['entity_percentage'] < 100, "Entity percentage should be less than 100%"
print("Basic verification passed.")

**Thought Questions:**
1. What percentage of tokens are entity tokens vs. O tokens? What problem does this class imbalance cause for training?
2. Are Chemical and Disease entities roughly equally represented? If not, what impact might this have on per-entity-type F1?
3. What is the average entity length in tokens? How does this inform your max_length choice?

### TODO 2: Tokenization Alignment

BERT uses WordPiece tokenization, which splits words into subword units. For NER, we need to carefully align the original word-level labels with the new subword tokens.

In [ ]:
def tokenize_and_align_labels(examples, tokenizer, max_length=128):
    """
    Tokenize text and align NER labels with WordPiece subword tokens.

    BERT's WordPiece tokenizer may split a single word into multiple subword
    tokens. For example, "myocardial" might become ["my", "##ocard", "##ial"].
    When this happens, we need to decide what label to assign to the subword
    tokens.

    Strategy: Assign the original word's label to the FIRST subword token.
    Assign -100 to all subsequent subword tokens (this tells PyTorch to
    ignore them in the loss computation).

    Args:
        examples: dict with 'tokens' (list of list of strings) and
                  'tags' (list of list of ints)
        tokenizer: BertTokenizerFast instance
        max_length: maximum sequence length (default 128)

    Returns:
        dict with 'input_ids', 'attention_mask', 'labels' fields
        ready for model training

    Hints:
    - Use tokenizer(examples['tokens'], is_split_into_words=True,
          truncation=True, max_length=max_length, padding='max_length')
    - Use tokenized_inputs.word_ids(batch_index=i) to get the mapping
      from subword tokens to original words
    - word_ids() returns None for special tokens ([CLS], [SEP], [PAD])
      -- assign -100 to these positions
    - For the first subword of each word, copy the original label
    - For subsequent subwords of the same word, assign -100

    Step-by-step:
    1. Tokenize all examples with is_split_into_words=True
    2. For each example in the batch:
       a. Get word_ids for the tokenized sequence
       b. Initialize labels list
       c. Track previous_word_id (starts as None)
       d. For each token position:
          - If word_id is None: append -100
          - If word_id != previous_word_id: append the original label
          - If word_id == previous_word_id: append -100
       e. Update previous_word_id
    3. Add 'labels' to the tokenized output dict
    """
    # YOUR CODE HERE
    pass

In [ ]:
# Verification: Test tokenization alignment
test_example = {
    'tokens': [['The', 'patient', 'has', 'hypertension']],
    'tags': [[0, 0, 0, 1]]  # Only 'hypertension' is an entity
}
result = tokenize_and_align_labels(test_example, tokenizer, max_length=16)
assert 'labels' in result, "Result must contain 'labels'"
assert len(result['labels'][0]) == 16, f"Labels length should be 16, got {len(result['labels'][0])}"
# Check that [CLS] and [SEP] positions have -100
assert result['labels'][0][0] == -100, "[CLS] token should have label -100"
print("Tokenization alignment verification passed.")
print(f"Input IDs: {result['input_ids'][0]}")
print(f"Labels:    {result['labels'][0]}")
tokens = tokenizer.convert_ids_to_tokens(result['input_ids'][0])
print(f"Tokens:    {tokens}")

---

## 3.2 Exploratory Data Analysis

Before building models, we need to understand the data distribution. This is especially important in NER because class imbalance (most tokens are O) and entity length distributions directly affect model performance.

### TODO 3: Visualize Entity Distributions

In [ ]:
def plot_entity_analysis(dataset, label_names):
    """
    Create a comprehensive EDA visualization with 4 subplots:

    1. Bar chart: Count of each entity type (B-Chemical, I-Chemical,
       B-Disease, I-Disease, O) in the training set
    2. Histogram: Distribution of sequence lengths (number of tokens
       per example)
    3. Histogram: Distribution of entity lengths (number of tokens per
       entity) -- separate colors for Chemical vs Disease
    4. Bar chart: Top 20 most frequent entity surface forms (the actual
       text of extracted entities)

    Args:
        dataset: HuggingFace dataset with 'tokens' and 'tags'
        label_names: list of label name strings

    Hints:
    - Use matplotlib with plt.subplots(2, 2, figsize=(14, 10))
    - To extract entity surface forms, iterate through tokens/tags
      and concatenate tokens that form each entity
    - Use Counter from collections for frequency counting
    - Add clear titles, axis labels, and a tight_layout()
    """
    # YOUR CODE HERE
    pass

plot_entity_analysis(dataset['train'], label_names)

**Thought Questions:**
1. Is the dataset balanced between Chemical and Disease entities? If not, should we use class-weighted loss?
2. What is the most common entity length? Does this inform your choice of max_length?
3. Are there any surprising entries in the top 20 entity list? (e.g., very short entities that might cause false positives)

---

## 3.3 Baseline Approach

Before training BERT, we establish a baseline using a simple dictionary-based approach -- similar to MedScribe's original rule-based system. This gives us a lower bound on performance and helps us appreciate the value of contextual representations.

### TODO 4: Dictionary-Based NER Baseline

In [ ]:
def build_entity_dictionary(dataset, label_names):
    """
    Build a simple entity dictionary from the training set.

    Extract all entity surface forms (e.g., "aspirin", "myocardial infarction")
    and their entity types from the training data. Store them in a dictionary
    mapping surface form -> most common entity type.

    Args:
        dataset: training split with 'tokens' and 'tags'
        label_names: list of label name strings

    Returns:
        dict mapping entity string (lowercased) -> entity type string

    Hints:
    - Iterate through each example's tokens and tags
    - When you encounter a B- tag, start collecting tokens for a new entity
    - Continue collecting while you see I- tags of the same type
    - When the entity ends, join the tokens and add to the dictionary
    - If an entity form appears with multiple types, keep the most common
    """
    # YOUR CODE HERE
    pass


def dictionary_ner(tokens, entity_dict):
    """
    Perform NER using simple dictionary lookup with longest-match.

    For each position in the token sequence, check if any subsequence
    starting at that position matches an entry in the entity dictionary.
    Use longest-match-first strategy.

    Args:
        tokens: list of token strings
        entity_dict: dict mapping entity string -> entity type

    Returns:
        list of BIO tag strings, same length as tokens

    Hints:
    - For each start position, try matching subsequences of decreasing
      length (longest match first)
    - When a match is found, assign B-{type} to the first token and
      I-{type} to subsequent tokens in the match
    - Skip positions that are already tagged (inside a previously
      matched entity)
    - Assign 'O' to all unmatched tokens
    """
    # YOUR CODE HERE
    pass

In [ ]:
from seqeval.metrics import classification_report, f1_score

def evaluate_baseline(dataset_split, entity_dict, label_names):
    """Evaluate the dictionary baseline on a dataset split."""
    true_labels = []
    pred_labels = []

    for example in dataset_split:
        tokens = example['tokens']
        true_tags = [label_names[t] for t in example['tags']]
        pred_tags = dictionary_ner(tokens, entity_dict)
        true_labels.append(true_tags)
        pred_labels.append(pred_tags)

    print("Dictionary Baseline Results:")
    print(classification_report(true_labels, pred_labels))
    return f1_score(true_labels, pred_labels)

# Build dictionary and evaluate
entity_dict = build_entity_dictionary(dataset['train'], label_names)
print(f"Dictionary size: {len(entity_dict)} entities")
baseline_f1 = evaluate_baseline(dataset['test'], entity_dict, label_names)
print(f"\nBaseline Entity-Level F1: {baseline_f1:.4f}")

**Thought Questions:**
1. What is the dictionary baseline F1? How does this compare to MedScribe's rule-based system (0.68)?
2. What types of entities does the dictionary approach miss? Why?
3. Can a dictionary approach ever handle negation? Why or why not?

---

## 3.4 Model Design

Now we build the BERT-based NER model. The architecture is straightforward: BERT-Base produces a 768-dimensional hidden state for each token, and we add a single linear classification layer that maps each hidden state to one of our entity labels.

The power comes from BERT's pre-trained representations. Each token's 768-dimensional hidden state already encodes rich contextual information because of the bidirectional self-attention mechanism. The classification head simply learns to map this rich representation to entity labels.

### TODO 5: Build the NER Model

In [ ]:
import torch.nn as nn
from transformers import BertModel

class BertForClinicalNER(nn.Module):
    """
    BERT-based model for clinical Named Entity Recognition.

    Architecture:
    1. BERT-Base encoder (768-dim hidden states, 12 layers, 12 heads)
    2. Dropout layer for regularization
    3. Linear classification head: 768 -> num_labels

    The classification head is applied independently to each token position.
    During training, tokens with label -100 are ignored in the loss computation.

    Args:
        num_labels: number of NER label classes (default: 11)
        dropout_rate: dropout probability (default: 0.1)
        model_name: pre-trained BERT model name (default: 'bert-base-uncased')

    Forward pass:
        Input: input_ids, attention_mask, labels (optional)
        Output: dict with 'logits' (always) and 'loss' (if labels provided)

    Hints:
    - Use BertModel.from_pretrained(model_name) to load BERT (NOT
      BertForTokenClassification -- build it yourself to understand
      the architecture)
    - The BERT output has shape (batch_size, seq_len, 768)
    - Apply dropout to the BERT output before the linear layer
    - Use nn.CrossEntropyLoss(ignore_index=-100) to handle masked tokens
    - The loss should be computed over ALL non-masked token positions

    Step-by-step for __init__:
    1. Load pre-trained BERT model
    2. Create dropout layer (nn.Dropout)
    3. Create linear classifier (nn.Linear: 768 -> num_labels)

    Step-by-step for forward:
    1. Pass input_ids and attention_mask through BERT
    2. Get the last hidden state from BERT output
    3. Apply dropout
    4. Pass through the linear classifier to get logits
    5. If labels are provided, compute cross-entropy loss
    6. Return dict with logits (and loss if applicable)
    """
    def __init__(self, num_labels=5, dropout_rate=0.1,
                 model_name='bert-base-uncased'):
        super().__init__()
        # YOUR CODE HERE
        pass

    def forward(self, input_ids, attention_mask, labels=None):
        # YOUR CODE HERE
        pass

In [ ]:
# Verification: Test your model
model = BertForClinicalNER(num_labels=len(label_names))
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Test forward pass with dummy input
dummy_input = tokenizer("The patient was diagnosed with diabetes mellitus",
                        return_tensors="pt", padding="max_length",
                        max_length=32, truncation=True)
dummy_labels = torch.zeros(1, 32, dtype=torch.long)
output = model(dummy_input['input_ids'], dummy_input['attention_mask'],
               labels=dummy_labels)

assert 'logits' in output, "Output must contain 'logits'"
assert 'loss' in output, "Output must contain 'loss' when labels are provided"
assert output['logits'].shape == (1, 32, len(label_names)), \
    f"Expected logits shape (1, 32, {len(label_names)}), got {output['logits'].shape}"
print("All assertions passed. Model architecture is correct.")

**Thought Questions:**
1. Why do we use `ignore_index=-100` in the loss function? What tokens does this exclude?
2. How many parameters are in the classification head vs. the BERT encoder? What does this ratio tell you about where the learning happens during fine-tuning?
3. What would happen if we froze the BERT encoder and only trained the classification head?

---

## 3.5 Training Strategy

Fine-tuning BERT requires careful hyperparameter choices. The pre-trained weights encode valuable language knowledge, and aggressive training can destroy this knowledge (catastrophic forgetting). We use a small learning rate, warm up gradually, and apply weight decay selectively.

**Why these choices:**
- **Learning rate 3e-5:** Small enough to preserve pre-trained knowledge, large enough to adapt to clinical text. The original BERT paper recommends 2e-5 to 5e-5 for fine-tuning.
- **AdamW over SGD:** AdamW handles the sparse gradients in NER well (most tokens are O, so gradients are sparse). It also decouples weight decay from the adaptive learning rate.
- **Linear warmup:** Prevents large early updates that could destroy pre-trained features. The model gradually "wakes up" to the new task.
- **Gradient clipping:** BERT's 12 transformer layers can produce large gradients during early training. Clipping at norm 1.0 prevents instability.

In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader

# Training hyperparameters
BATCH_SIZE = 16
LEARNING_RATE = 3e-5
NUM_EPOCHS = 5
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
MAX_GRAD_NORM = 1.0

# Prepare tokenized datasets
tokenized_train = dataset['train'].map(
    lambda x: tokenize_and_align_labels(x, tokenizer, max_length=128),
    batched=True, remove_columns=dataset['train'].column_names
)
tokenized_val = dataset['validation'].map(
    lambda x: tokenize_and_align_labels(x, tokenizer, max_length=128),
    batched=True, remove_columns=dataset['validation'].column_names
)

tokenized_train.set_format('torch')
tokenized_val.set_format('torch')

train_loader = DataLoader(tokenized_train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(tokenized_val, batch_size=BATCH_SIZE)

# Setup optimizer with weight decay (exclude bias and LayerNorm)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)],
     'weight_decay': WEIGHT_DECAY},
    {'params': [p for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)],
     'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE)

total_steps = len(train_loader) * NUM_EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Training on: {device}")
print(f"Total training steps: {total_steps}")
print(f"Warmup steps: {warmup_steps}")

### TODO 6: Training and Evaluation Loops

In [ ]:
from seqeval.metrics import classification_report as seq_classification_report
from seqeval.metrics import f1_score as seq_f1_score

def train_epoch(model, train_loader, optimizer, scheduler, device,
                max_grad_norm=1.0):
    """
    Train the model for one epoch.

    Args:
        model: BertForClinicalNER instance
        train_loader: DataLoader for training data
        optimizer: AdamW optimizer
        scheduler: learning rate scheduler
        device: torch device (cuda or cpu)
        max_grad_norm: maximum gradient norm for clipping

    Returns:
        float: average training loss for the epoch

    Hints:
    - Set model to training mode with model.train()
    - For each batch:
      1. Move input_ids, attention_mask, labels to device
      2. Zero the gradients
      3. Forward pass through the model
      4. Get the loss from the output
      5. Backward pass (loss.backward())
      6. Clip gradients with torch.nn.utils.clip_grad_norm_
      7. Update weights (optimizer.step())
      8. Update learning rate (scheduler.step())
      9. Accumulate the loss for logging
    - Return the average loss across all batches
    - Print progress every 50 batches (batch loss and learning rate)
    """
    # YOUR CODE HERE
    pass


def evaluate(model, val_loader, device, label_names):
    """
    Evaluate the model on a validation/test set.

    Args:
        model: BertForClinicalNER instance
        val_loader: DataLoader for validation/test data
        device: torch device
        label_names: list of label name strings

    Returns:
        tuple: (average_loss, entity_f1, classification_report_string)

    Hints:
    - Set model to evaluation mode with model.eval()
    - Use torch.no_grad() context manager
    - For each batch:
      1. Forward pass to get logits and loss
      2. Get predictions via argmax over the label dimension
      3. Convert predictions and true labels to label name strings
      4. IMPORTANT: Skip tokens where the true label is -100
         (subword tokens, special tokens, padding)
    - Use seqeval.metrics.f1_score and classification_report
    - Collect predictions at the SEQUENCE level (list of list of
      label strings) for seqeval
    """
    # YOUR CODE HERE
    pass

In [ ]:
import time

best_f1 = 0.0
train_losses = []
val_f1_scores = []

for epoch in range(NUM_EPOCHS):
    start_time = time.time()

    # Train
    train_loss = train_epoch(model, train_loader, optimizer, scheduler,
                             device, MAX_GRAD_NORM)
    train_losses.append(train_loss)

    # Evaluate
    val_loss, val_f1, report = evaluate(model, val_loader, device, label_names)
    val_f1_scores.append(val_f1)

    epoch_time = time.time() - start_time

    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS} ({epoch_time:.1f}s)")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Val Loss:   {val_loss:.4f}")
    print(f"  Val F1:     {val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), 'best_model.pt')
        print(f"  New best model saved (F1: {best_f1:.4f})")

    print("\n" + report)

print(f"\nTraining complete. Best validation F1: {best_f1:.4f}")

---

## 3.6 Evaluation

Now we perform a comprehensive evaluation comparing BERT's performance against the dictionary baseline.

### TODO 7: Comprehensive Evaluation

In [ ]:
def comprehensive_evaluation(model, test_loader, device, label_names,
                             baseline_f1, train_losses, val_f1_scores):
    """
    Perform a thorough evaluation comparing BERT to the baseline.

    Generate the following:
    1. Entity-level classification report (precision, recall, F1 per type)
    2. Confusion matrix for token-level predictions (use sklearn)
    3. A comparison bar chart: Baseline F1 vs BERT F1 for each entity type
    4. Training curves plot: train loss and validation F1 vs epoch

    Args:
        model: trained BertForClinicalNER (load best_model.pt weights)
        test_loader: DataLoader for test data
        device: torch device
        label_names: list of label name strings
        baseline_f1: float, the dictionary baseline F1 score
        train_losses: list of per-epoch training losses
        val_f1_scores: list of per-epoch validation F1 scores

    Returns:
        dict with 'overall_f1', 'per_type_f1', 'improvement_over_baseline'

    Hints:
    - Load best model weights: model.load_state_dict(torch.load('best_model.pt'))
    - Use seqeval for entity-level metrics
    - Use sklearn.metrics.confusion_matrix for token-level confusion
    - Use matplotlib for all plots with plt.subplots(2, 2, figsize=(14, 10))
    - For the comparison chart, you will need to compute per-entity-type
      F1 for the dictionary baseline as well
    """
    # YOUR CODE HERE
    pass

# Prepare test data
tokenized_test = dataset['test'].map(
    lambda x: tokenize_and_align_labels(x, tokenizer, max_length=128),
    batched=True, remove_columns=dataset['test'].column_names
)
tokenized_test.set_format('torch')
test_loader = DataLoader(tokenized_test, batch_size=BATCH_SIZE)

results = comprehensive_evaluation(model, test_loader, device, label_names,
                                   baseline_f1, train_losses, val_f1_scores)
print(f"\nBERT Entity-Level F1: {results['overall_f1']:.4f}")
print(f"Baseline F1: {baseline_f1:.4f}")
print(f"Improvement: {results['improvement_over_baseline']:.4f}")

**Thought Questions:**
1. Which entity type benefits most from BERT? Why might BERT help more for some entity types than others?
2. Is there a significant gap between precision and recall for any entity type? What does this tell you about the model's behavior?
3. Does the training loss curve suggest overfitting? How would you tell?

---

## 3.7 Error Analysis

Understanding where the model fails is as important as knowing where it succeeds. In a clinical setting, certain errors are more dangerous than others -- missing a diagnosis is worse than a boundary error.

### TODO 8: Systematic Error Analysis

In [ ]:
def error_analysis(model, dataset_split, tokenizer, device, label_names,
                   num_examples=100):
    """
    Perform systematic error analysis on model predictions.

    Analyze the first num_examples from the dataset and categorize errors
    into the following types:
    1. Boundary errors: entity detected but span boundaries are wrong
    2. Type errors: entity span is correct but type is wrong
    3. Missing entities: true entity completely missed (false negative)
    4. Hallucinated entities: entity predicted where none exists (false positive)
    5. Negation errors: entity in a negated context incorrectly tagged
       (check for negation trigger words within 5 tokens)

    For each error category, collect and print:
    - Count and percentage of total errors
    - 3 representative examples showing the token sequence, true labels,
      and predicted labels (highlight the error region)

    Args:
        model: trained BertForClinicalNER
        dataset_split: dataset split to analyze
        tokenizer: BertTokenizerFast instance
        device: torch device
        label_names: list of label name strings
        num_examples: number of examples to analyze

    Returns:
        dict mapping error_type -> list of error instances

    Hints:
    - For each example, compare predicted entity spans to true entity spans
    - An entity span is defined by its start index, end index, and type
    - Boundary error: overlap between predicted and true span, but not exact
    - Use negation triggers: ['no', 'not', 'without', 'denies', 'denied',
      'negative', 'absent', 'ruled out', 'unlikely']
    - Format examples clearly with aligned columns for readability
    """
    # YOUR CODE HERE
    pass

errors = error_analysis(model, dataset['test'], tokenizer, device, label_names)

**Thought Questions:**
1. Which error category is most common? What does this suggest about the model's weaknesses?
2. Are there patterns in the missing entities? (e.g., are they unusually long, rare, or ambiguous?)
3. How would you prioritize fixing these errors in a production clinical system? Which are most clinically dangerous?

---

## 3.8 Inference Optimization

MedScribe requires code suggestions within 3 seconds of a note being finalized. Let us profile our model's inference performance and explore optimization strategies.

### TODO 9: Latency Profiling

In [ ]:
def profile_inference(model, tokenizer, device, sample_texts=None):
    """
    Profile model inference latency and explore optimization strategies.

    Steps:
    1. Measure baseline inference latency on sample clinical texts
       (average over 100 runs, report mean, p50, p95, p99)
    2. Compare latency for different sequence lengths (32, 64, 128, 256, 512)
    3. Measure the effect of batch size on throughput (notes/second)
    4. Implement and measure dynamic padding (pad to longest in batch,
       not max_length) -- report the speedup

    Args:
        model: trained BertForClinicalNER
        tokenizer: BertTokenizerFast
        device: torch device
        sample_texts: optional list of sample clinical texts. If None,
            use the following defaults:
            [
                "Patient presents with acute chest pain radiating to left arm.",
                "Prescribed metformin 500mg twice daily for type 2 diabetes.",
                "CT scan of abdomen shows no evidence of appendicitis.",
                "History of hypertension, currently on lisinopril 10mg.",
            ]

    Returns:
        dict with latency statistics and throughput measurements

    Hints:
    - Use torch.cuda.synchronize() before timing on GPU
    - Use time.perf_counter() for high-resolution timing
    - For dynamic padding, tokenize the batch without max_length,
      then pad to the longest sequence in the batch
    - Calculate throughput as: batch_size / batch_latency
    - Create a line plot: sequence length vs latency
    - Create a bar chart: batch size vs throughput

    MedScribe's requirement: p95 latency < 500ms per note.
    Does your model meet this requirement?
    """
    # YOUR CODE HERE
    pass

latency_stats = profile_inference(model, tokenizer, device)

**Thought Questions:**
1. How does inference latency scale with sequence length? Is it linear or quadratic? Why? (Hint: think about the self-attention computation.)
2. What is the optimal batch size for throughput on your hardware? Is there a point of diminishing returns?
3. Does dynamic padding provide meaningful speedup? In what scenarios would it matter most?

---

## 3.9 Ethical and Regulatory Analysis

Deploying an NLP model in a clinical setting carries significant ethical responsibilities. Errors can lead to wrong diagnoses, incorrect treatments, and harm to patients. Before deploying this model, we must carefully assess its biases, calibration, and failure modes.

### TODO 10: Clinical AI Ethics Assessment

In [ ]:
def ethics_assessment(model, tokenizer, device, label_names):
    """
    Conduct an ethical impact assessment for deploying this NER model
    in a clinical setting.

    Implement the following analyses:

    1. Demographic bias test: Run the model on matched sentence pairs
       that differ only in demographic terms. For example:
       - "The [male/female] patient was diagnosed with depression."
       - "The [young/elderly] patient has chronic pain."
       Compare entity predictions across demographic variants. Any
       difference indicates potential bias.

    2. Confidence calibration: For all predictions on the test set,
       bin predictions by confidence (softmax probability) and compute
       accuracy within each bin. Plot a reliability diagram. A well-
       calibrated model should have accuracy ~= confidence in each bin.

    3. Failure mode severity ranking: For each error type from
       error_analysis, assign a clinical severity score:
       - Critical (3): missed diagnosis, wrong medication
       - Moderate (2): wrong anatomy, missed lab test
       - Minor (1): boundary error, partial entity

       Report the distribution of error severities.

    Print a summary with:
    - Whether demographic bias was detected (yes/no for each pair)
    - Expected Calibration Error (ECE)
    - Distribution of error severities
    - A brief recommendation: is this model safe to deploy? What
      guardrails are needed?

    Args:
        model: trained BertForClinicalNER
        tokenizer: BertTokenizerFast
        device: torch device
        label_names: list of label name strings

    Returns:
        dict with 'bias_results', 'ece', 'severity_distribution',
             'deployment_recommendation'

    Hints:
    - For bias testing, create pairs of sentences that are identical
      except for the demographic term
    - For calibration, use torch.softmax on logits and take the max
      probability as the confidence score
    - Use 10 equally-spaced bins for the reliability diagram
    - ECE below 0.05 is considered well-calibrated
    """
    # YOUR CODE HERE
    pass

ethics_results = ethics_assessment(model, tokenizer, device, label_names)

**Thought Questions:**
1. Did you detect demographic bias? If so, what is the most likely cause and how would you mitigate it?
2. Is the model well-calibrated? Why does calibration matter in a clinical setting? (Consider: a doctor sees "confidence: 0.90" -- what should that mean?)
3. Based on your error severity analysis, what specific guardrails would you recommend for production deployment?

---

## Summary

In this notebook, you built a complete clinical NER pipeline:

1. **Data exploration**: Analyzed the BC5CDR biomedical NER dataset and understood its class imbalance and entity distributions
2. **Tokenization alignment**: Solved the critical problem of aligning word-level NER labels with BERT's subword tokens
3. **Dictionary baseline**: Built a rule-based NER system (similar to MedScribe's legacy system) and quantified its limitations
4. **BERT NER model**: Constructed a BERT-based token classifier from scratch (not using a pre-built class)
5. **Fine-tuning**: Trained the model with careful hyperparameter choices to prevent catastrophic forgetting
6. **Evaluation**: Compared BERT against the baseline with entity-level F1 and per-type analysis
7. **Error analysis**: Categorized model failures to understand where clinical risk lies
8. **Latency profiling**: Verified the model meets MedScribe's 500ms inference budget
9. **Ethics assessment**: Tested for demographic bias, calibration, and clinical error severity

For next steps, read **Section 4** of the case study document (Production and System Design Extension) to understand how this model would be deployed at scale across MedScribe's 12 hospital clients, including API design, monitoring, drift detection, and A/B testing.

In [ ]:
#@title 💬 AI Teaching Assistant — Click ▶ to start
#@markdown This AI chatbot reads your notebook and can answer questions about any concept, code, or exercise.

import json as _json
import requests as _requests
from google.colab import output as _output
from IPython.display import display, HTML as _HTML, Markdown as _Markdown

# --- Read notebook content for context ---
def _get_notebook_context():
    try:
        from google.colab import _message
        nb = _message.blocking_request("get_ipynb", request="", timeout_sec=10)
        cells = nb.get("ipynb", {}).get("cells", [])
        parts = []
        for cell in cells:
            src = "".join(cell.get("source", []))
            tags = cell.get("metadata", {}).get("tags", [])
            if "chatbot" in tags:
                continue
            if src.strip():
                ct = cell.get("cell_type", "unknown")
                parts.append(f"[{ct.upper()}]\n{src}")
        return "\n\n---\n\n".join(parts)
    except Exception:
        return "Notebook content unavailable."

_NOTEBOOK_CONTEXT = _get_notebook_context()
_CHAT_HISTORY = []
_API_URL = "https://course-creator-brown.vercel.app/api/chat"

def _notebook_chat(question):
    global _CHAT_HISTORY
    try:
        resp = _requests.post(_API_URL, json={
            'question': question,
            'context': _NOTEBOOK_CONTEXT[:100000],
            'history': _CHAT_HISTORY[-10:],
        }, timeout=60)
        data = resp.json()
        answer = data.get('answer', 'Sorry, I could not generate a response.')
        _CHAT_HISTORY.append({'role': 'user', 'content': question})
        _CHAT_HISTORY.append({'role': 'assistant', 'content': answer})
        return answer
    except Exception as e:
        return f'Error connecting to teaching assistant: {str(e)}'

_output.register_callback('notebook_chat', _notebook_chat)

def ask(question):
    """Ask the AI teaching assistant a question about this notebook."""
    answer = _notebook_chat(question)
    display(_Markdown(answer))

print("\u2705 AI Teaching Assistant is ready!")
print("\U0001f4a1 Use the chat below, or call ask(\'your question\') in any cell.")

# --- Display chat widget ---
display(_HTML('''<style>
  .vc-wrap{font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,sans-serif;max-width:100%;border-radius:16px;overflow:hidden;box-shadow:0 4px 24px rgba(0,0,0,.12);background:#fff;border:1px solid #e5e7eb}
  .vc-hdr{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;padding:16px 20px;display:flex;align-items:center;gap:12px}
  .vc-avatar{width:42px;height:42px;background:rgba(255,255,255,.2);border-radius:50%;display:flex;align-items:center;justify-content:center;font-size:22px}
  .vc-hdr h3{font-size:16px;font-weight:600;margin:0}
  .vc-hdr p{font-size:12px;opacity:.85;margin:2px 0 0}
  .vc-msgs{height:420px;overflow-y:auto;padding:16px;background:#f8f9fb;display:flex;flex-direction:column;gap:10px}
  .vc-msg{display:flex;flex-direction:column;animation:vc-fade .25s ease}
  .vc-msg.user{align-items:flex-end}
  .vc-msg.bot{align-items:flex-start}
  .vc-bbl{max-width:85%;padding:10px 14px;border-radius:16px;font-size:14px;line-height:1.55;word-wrap:break-word}
  .vc-msg.user .vc-bbl{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border-bottom-right-radius:4px}
  .vc-msg.bot .vc-bbl{background:#fff;color:#1a1a2e;border:1px solid #e8e8e8;border-bottom-left-radius:4px}
  .vc-bbl code{background:rgba(0,0,0,.07);padding:2px 6px;border-radius:4px;font-size:13px;font-family:'Fira Code',monospace}
  .vc-bbl pre{background:#1e1e2e;color:#cdd6f4;padding:12px;border-radius:8px;overflow-x:auto;margin:8px 0;font-size:13px}
  .vc-bbl pre code{background:none;padding:0;color:inherit}
  .vc-bbl h3,.vc-bbl h4{margin:10px 0 4px;font-size:15px}
  .vc-bbl ul,.vc-bbl ol{margin:4px 0;padding-left:20px}
  .vc-bbl li{margin:2px 0}
  .vc-chips{display:flex;flex-wrap:wrap;gap:8px;padding:0 16px 12px;background:#f8f9fb}
  .vc-chip{background:#fff;border:1px solid #d1d5db;border-radius:20px;padding:6px 14px;font-size:12px;cursor:pointer;transition:all .15s;color:#4b5563}
  .vc-chip:hover{border-color:#667eea;color:#667eea;background:#f0f0ff}
  .vc-input{display:flex;padding:12px 16px;background:#fff;border-top:1px solid #eee;gap:8px}
  .vc-input input{flex:1;padding:10px 16px;border:2px solid #e8e8e8;border-radius:24px;font-size:14px;outline:none;transition:border-color .2s}
  .vc-input input:focus{border-color:#667eea}
  .vc-input button{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border:none;border-radius:50%;width:42px;height:42px;cursor:pointer;display:flex;align-items:center;justify-content:center;font-size:18px;transition:transform .1s}
  .vc-input button:hover{transform:scale(1.05)}
  .vc-input button:disabled{opacity:.5;cursor:not-allowed;transform:none}
  .vc-typing{display:flex;gap:5px;padding:4px 0}
  .vc-typing span{width:8px;height:8px;background:#667eea;border-radius:50%;animation:vc-bounce 1.4s infinite ease-in-out}
  .vc-typing span:nth-child(2){animation-delay:.2s}
  .vc-typing span:nth-child(3){animation-delay:.4s}
  @keyframes vc-bounce{0%,80%,100%{transform:scale(0)}40%{transform:scale(1)}}
  @keyframes vc-fade{from{opacity:0;transform:translateY(8px)}to{opacity:1;transform:translateY(0)}}
  .vc-note{text-align:center;font-size:11px;color:#9ca3af;padding:8px 16px 12px;background:#fff}
</style>
<div class="vc-wrap">
  <div class="vc-hdr">
    <div class="vc-avatar">&#129302;</div>
    <div>
      <h3>Vizuara Teaching Assistant</h3>
      <p>Ask me anything about this notebook</p>
    </div>
  </div>
  <div class="vc-msgs" id="vcMsgs">
    <div class="vc-msg bot">
      <div class="vc-bbl">&#128075; Hi! I've read through this entire notebook. Ask me about any concept, code block, or exercise &mdash; I'm here to help you learn!</div>
    </div>
  </div>
  <div class="vc-chips" id="vcChips">
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Explain the main concept</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Help with the TODO exercise</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Summarize what I learned</span>
  </div>
  <div class="vc-input">
    <input type="text" id="vcIn" placeholder="Ask about concepts, code, exercises..." />
    <button id="vcSend" onclick="vcSendMsg()">&#10148;</button>
  </div>
  <div class="vc-note">AI-generated &middot; Verify important information &middot; <a href="#" onclick="vcClear();return false" style="color:#667eea">Clear chat</a></div>
</div>
<script>
(function(){
  var msgs=document.getElementById('vcMsgs'),inp=document.getElementById('vcIn'),
      btn=document.getElementById('vcSend'),chips=document.getElementById('vcChips');

  function esc(s){var d=document.createElement('div');d.textContent=s;return d.innerHTML}

  function md(t){
    return t
      .replace(/```(\w*)\n([\s\S]*?)```/g,function(_,l,c){return '<pre><code>'+esc(c)+'</code></pre>'})
      .replace(/`([^`]+)`/g,'<code>$1</code>')
      .replace(/\*\*([^*]+)\*\*/g,'<strong>$1</strong>')
      .replace(/\*([^*]+)\*/g,'<em>$1</em>')
      .replace(/^#### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^## (.+)$/gm,'<h3>$1</h3>')
      .replace(/^\d+\. (.+)$/gm,'<li>$1</li>')
      .replace(/^- (.+)$/gm,'<li>$1</li>')
      .replace(/\n\n/g,'<br><br>')
      .replace(/\n/g,'<br>');
  }

  function addMsg(text,isUser){
    var m=document.createElement('div');m.className='vc-msg '+(isUser?'user':'bot');
    var b=document.createElement('div');b.className='vc-bbl';
    b.innerHTML=isUser?esc(text):md(text);
    m.appendChild(b);msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function showTyping(){
    var m=document.createElement('div');m.className='vc-msg bot';m.id='vcTyping';
    m.innerHTML='<div class="vc-bbl"><div class="vc-typing"><span></span><span></span><span></span></div></div>';
    msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function hideTyping(){var e=document.getElementById('vcTyping');if(e)e.remove()}

  window.vcSendMsg=function(){
    var q=inp.value.trim();if(!q)return;
    inp.value='';chips.style.display='none';
    addMsg(q,true);showTyping();btn.disabled=true;
    google.colab.kernel.invokeFunction('notebook_chat',[q],{})
      .then(function(r){
        hideTyping();
        var a=r.data['application/json'];
        addMsg(typeof a==='string'?a:JSON.stringify(a),false);
      })
      .catch(function(){
        hideTyping();
        addMsg('Sorry, I encountered an error. Please check your internet connection and try again.',false);
      })
      .finally(function(){btn.disabled=false;inp.focus()});
  };

  window.vcAsk=function(q){inp.value=q;vcSendMsg()};
  window.vcClear=function(){
    msgs.innerHTML='<div class="vc-msg bot"><div class="vc-bbl">&#128075; Chat cleared. Ask me anything!</div></div>';
    chips.style.display='flex';
  };

  inp.addEventListener('keypress',function(e){if(e.key==='Enter')vcSendMsg()});
  inp.focus();
})();
</script>'''))